In [1]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
import numpy as np
from sqlalchemy import create_engine
!pip install pymysql

In [2]:
host = "hotel-reservations-dataset.c7okku88e20j.us-east-1.rds.amazonaws.com"
port = 3306
dbname = "hotel"
user = "admin"
password = "Hotel#0405"

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}')

query = "SELECT * FROM dataset"
df = pd.read_sql(query, engine)
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,booking_status,label_avg_price_room
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,0,Not_Canceled,1
1,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,1,Not_Canceled,2
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,0,Canceled,1
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,0,Canceled,2
4,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,0,Canceled,2


In [3]:
df.isnull().sum()

no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
no_of_special_requests                  0
booking_status                          0
label_avg_price_room                    0
dtype: int64

In [4]:
df_dummies = pd.get_dummies(df, columns=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 'booking_status'])
df_dummies.columns

Index(['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests', 'label_avg_price_room',
       'type_of_meal_plan_Meal Plan 1', 'type_of_meal_plan_Meal Plan 2',
       'type_of_meal_plan_Meal Plan 3', 'type_of_meal_plan_Not Selected',
       'room_type_reserved_Room_Type 1', 'room_type_reserved_Room_Type 2',
       'room_type_reserved_Room_Type 3', 'room_type_reserved_Room_Type 4',
       'room_type_reserved_Room_Type 5', 'room_type_reserved_Room_Type 6',
       'room_type_reserved_Room_Type 7', 'market_segment_type_Aviation',
       'market_segment_type_Complementary', 'market_segment_type_Corporate',
       'market_segment_type_Offline', 'market_segment_type_Online',
       'booking_status_Canceled', 'booking_status_Not_Canceled']

In [5]:
X = df_dummies.drop(columns=['label_avg_price_room'])
X.columns

Index(['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests', 'type_of_meal_plan_Meal Plan 1',
       'type_of_meal_plan_Meal Plan 2', 'type_of_meal_plan_Meal Plan 3',
       'type_of_meal_plan_Not Selected', 'room_type_reserved_Room_Type 1',
       'room_type_reserved_Room_Type 2', 'room_type_reserved_Room_Type 3',
       'room_type_reserved_Room_Type 4', 'room_type_reserved_Room_Type 5',
       'room_type_reserved_Room_Type 6', 'room_type_reserved_Room_Type 7',
       'market_segment_type_Aviation', 'market_segment_type_Complementary',
       'market_segment_type_Corporate', 'market_segment_type_Offline',
       'market_segment_type_Online', 'booking_status_Canceled',
       'booking_status_Not_Canceled'],
      dtype='object')

In [6]:
y = df_dummies['label_avg_price_room']

In [20]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Configurações SageMaker

In [9]:
import sagemaker
import boto3
from sagemaker import Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [10]:
session = sagemaker.Session()
bucket = 'sprint4-5'
model_folder = 'modelos/hotel_reservation/XGBoost'
subfolder = 'datasets/hotel_reservation' 
key_train = 'train-data-xgboost'
key_test = 'test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = f's3://{bucket}/{subfolder}/train/{key_train}'
s3_test_data = f's3://{bucket}/{subfolder}/test/{key_test}'
output_location = f's3://{bucket}/{model_folder}/output'
s3 = boto3.client('s3')
print(f'Role: {role}')
print(f'Localização da base de treinamento: {s3_train_data}')
print(f'Localização da base de teste: {s3_test_data}')
print(f'Modelo final será salvo em: {output_location}')

Role: arn:aws:iam::654654631603:role/service-role/AmazonSageMaker-ExecutionRole-20240913T151513
Localização da base de treinamento: s3://sprint4-5/datasets/hotel_reservation/train/train-data-xgboost
Localização da base de teste: s3://sprint4-5/datasets/hotel_reservation/test/test-data-xgboost
Modelo final será salvo em: s3://sprint4-5/modelos/hotel_reservation/XGBoost/output


In [11]:
import os

test_data = np.concatenate((y_test.reshape(-1, 1), X_test), axis=1)
train_data = np.concatenate((y_train.reshape(-1, 1), X_train), axis=1)

np.savetxt('hotel-reservation-train-data-xgboost.csv', train_data, delimiter=',', fmt='%f')
np.savetxt('hotel-reservation-test-data-xgboost.csv', test_data, delimiter=',', fmt='%f')

with open('hotel-reservation-train-data-xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subfolder, 'train', key_train)).upload_fileobj(f)

In [12]:
with open('hotel-reservation-test-data-xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subfolder, 'test', key_test)).upload_fileobj(f)

# Treinamento do modelo

In [13]:
from sagemaker import image_uris

container = image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.7-1')

In [14]:
xgboost = sagemaker.estimator.Estimator(image_uri=container, role=role, instance_count=1, instance_type='ml.m5.2xlarge', output_path=output_location, sagemaker_session=session)

In [21]:
xgboost.set_hyperparameters(num_round=100, objective='reg:squarederror')

In [22]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type='S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type='S3Prefix')

data_channels = {'train': train_input, 'validation': validation_input}

In [23]:
xgboost.fit(data_channels)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-15-23-49-58-739


2024-09-15 23:50:00 Starting - Starting the training job...
2024-09-15 23:50:14 Starting - Preparing the instances for training...
2024-09-15 23:50:58 Downloading - Downloading the training image......
2024-09-15 23:51:44 Training - Training image download completed. Training in progress..[2024-09-15 23:52:05.422 ip-10-2-113-14.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-15 23:52:05.447 ip-10-2-113-14.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-09-15:23:52:05:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-15:23:52:05:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-09-15:23:52:05:INFO] No GPUs detected (normal if no gpus installed)
[2024-09-15:23:52:05:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-09-15:23:52:05:INFO] Determined 0 GPU(s) available on the instance.
[2024-09-15:23:52:05:INFO] Determined delimiter of CS

# Avaliação

In [24]:
xgboost_regressor = xgboost.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-15-23-53-11-339
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-09-15-23-53-11-339
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-09-15-23-53-11-339


-----------!

In [25]:
from sagemaker.serializers import CSVSerializer

xgboost_regressor.serializer = CSVSerializer()

In [26]:
X_test.shape, type(X_test)

((10883, 31), numpy.ndarray)

In [27]:
previsoes = np.array(xgboost_regressor.predict(X_test).decode('utf-8').split('\n'))
previsoes = [x for x in previsoes if x]
previsoes = np.array(previsoes).astype(np.float32)
previsoes

array([2.5652454, 1.3122922, 1.8439097, ..., 1.5989388, 2.9506352,
       2.0021849], dtype=float32)

In [28]:
previsoes.shape, y_test.shape

((10883,), (10883,))

In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, previsoes)
mse = mean_squared_error(y_test, previsoes)
print(f'MAE = {mae}\nMSE = {mse}')

MAE = 0.28211843967437744
MSE = 0.1585792452096939


# Tuning

In [30]:
tuning_job_config = {
    "ParameterRanges" : {
        "CategoricalParameterRanges": [],
        "ContinuousParameterRanges": [
            {
                "MaxValue": "1",
                "MinValue": "0",
                "Name": "eta"
            },
            {
                "MaxValue": "2",
                "MinValue": "0",
                "Name": "alpha"
            },
            {
                "MaxValue": "10",
                "MinValue": "1",
                "Name": "min_child_weight"
            }
        ],
        "IntegerParameterRanges": [
            {
                "MaxValue": "10",
                "MinValue": "1",
                "Name": "max_depth"
            }
        ],
        "IntegerParameterRanges": [
            {
                "MaxValue": "300",
                "MinValue": "50",
                "Name": "num_round"
            }
        ]
    },
    "ResourceLimits": {
        "MaxNumberOfTrainingJobs": 9,
        "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
        "MetricName": "validation:rmse",
        "Type": "Minimize"
    }
}

In [31]:
training_job_definition = {
    "AlgorithmSpecification": {
        "TrainingImage": container, 
        "TrainingInputMode": "File"
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_train_data
                }
            }
        },
        {
            "ChannelName": "validation",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_test_data
                }
            }
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": f's3://{bucket}/{subfolder}/output'
    },
    "ResourceConfig": {
        "InstanceCount": 2,
        "InstanceType": "ml.c4.2xlarge",
        "VolumeSizeInGB": 10
    },
    "RoleArn": role,
    "StaticHyperParameters": {
        "eval_metric": "rmse",
        "objective": "reg:linear",
        "rate_drop": "0.3",
        "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 43200
    }
}

In [33]:
smclient = boto3.client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = 'xgboosttuning-hotel', HyperParameterTuningJobConfig = tuning_job_config, TrainingJobDefinition = training_job_definition)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:654654631603:hyper-parameter-tuning-job/xgboosttuning-hotel',
 'ResponseMetadata': {'RequestId': '3d03c336-1d6d-4f5e-97af-9df2293ca8fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3d03c336-1d6d-4f5e-97af-9df2293ca8fb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '120',
   'date': 'Mon, 16 Sep 2024 00:00:19 GMT'},
  'RetryAttempts': 0}}

# Modelo otimizado

In [36]:
container = image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.7-1')
xgboostTuning = sagemaker.estimator.Estimator(image_uri=container, role=role, instance_count=1, instance_type='ml.m5.2xlarge', output_path=output_location, sagemaker_session=session)
xgboostTuning.set_hyperparameters(num_round=211, eta=0.5295033866583916, min_child_weight=4.108690686338631, alpha=1.974355903913699, tweedie_variance_power=1.4, rate_drop=	0.3 ,objective='reg:squarederror')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [37]:
xgboostTuning.fit(data_channels)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-16-00-18-36-300


2024-09-16 00:18:37 Starting - Starting the training job...
2024-09-16 00:18:51 Starting - Preparing the instances for training...
2024-09-16 00:19:34 Downloading - Downloading the training image......
2024-09-16 00:20:20 Training - Training image download completed. Training in progress..[2024-09-16 00:20:41.092 ip-10-2-100-149.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-16 00:20:41.113 ip-10-2-100-149.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-09-16:00:20:41:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-16:00:20:41:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-09-16:00:20:41:INFO] No GPUs detected (normal if no gpus installed)
[2024-09-16:00:20:41:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-09-16:00:20:41:INFO] Determined 0 GPU(s) available on the instance.
[2024-09-16:00:20:41:INFO] Determined delimiter of 

In [44]:
xgboost_regressor_tuning = xgboost.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-16-00-29-01-065
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-09-16-00-29-01-065
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-09-16-00-29-01-065


-------!

In [45]:
xgboost_regressor_tuning.serializer = CSVSerializer()

In [46]:
previsoes = np.array(xgboost_regressor_tuning.predict(X_test).decode('utf-8').split('\n'))
previsoes = [x for x in previsoes if x]
previsoes = np.array(previsoes).astype(np.float32)
previsoes

array([2.5652454, 1.3122922, 1.8439097, ..., 1.5989388, 2.9506352,
       2.0021849], dtype=float32)